---
title: "From ChatBot to Agentic AI"
execute:
    enabled: true
---

::: {.callout-note appearance="simple"}
**Spoiler**: Agents don't "think" in the human sense. They loop. They are state machines that use an LLM to decide the next transition.
:::

## The Mechanism

![ReAct Loop](../figs/react.png){width="70%" fig-align="center"}

The naive view is that agents are "smarter" chatbots. They aren't. They're a core component wrapped in a control loop. A chatbot generates text and stops. An agent generates text, parses it for actionable commands, executes those commands in the real world, observes the results, and feeds those results back into the next prompt. The intelligence doesn't come from the model---it comes from the **feedback loop**.

This is the **ReAct Pattern** (Reason + Act). A standard chatbot is a pure function: $\text{Output} = \text{Model}(\text{Input})$. An agent is a state machine:

```python
while not task_complete:
    observation = get_environment_state()
    thought = model(observation)
    action = parse_action(thought)
    result = execute(action)
    observation = result  # Feedback loop
```

The critical insight is the feedback loop. If the agent tries to import a missing library (Action) and receives `ModuleNotFoundError` (Observation), the next iteration's Thought will be "I need to install this library," rather than hallucinating success. The model corrects itself not through introspection, but through collision with reality.

ReAct framework is proposed by [@yao2022react]. The core idea is to prompt the LLM to generate both reasoning traces and task-specific actions in an interleaved manner. Specifically, the prompt structure follows a sequence: `Thought` $\rightarrow$ `Action` $\rightarrow$ `Observation`.

1.  **Thought**: The model reasons about the current state and what needs to be done.
2.  **Action**: The model outputs a specific command to interact with an external environment (e.g., `Search[Apple]`).
3.  **Observation**: The environment executes the action and returns the result (e.g., search results for "Apple").

This cycle repeats until the task is solved.

## The ReAct Framework with `langgraph`

Let's build an agent that can explore and analyze a real dataset. We'll use **LangGraph**—a framework from LangChain that models agents as state machines. Unlike simple loops, LangGraph lets you define explicit control flow: decision nodes, parallel execution, conditional branching, and state persistence.


::: {.callout-note}

Install LangGraph and LangChain:

```bash
pip install langgraph langchain langchain-ollama
```

:::

### Creating a Tool: A Fish Market Dataset

Let's build an agent that can explore and analyze a real dataset. We'll use the Fish Market dataset from Hugging Face—a collection of measurements from different fish species. First, load the data:

In [ ]:
import pandas as pd

# Load the Fish dataset
df = pd.read_csv("hf://datasets/scikit-learn/Fish/Fish.csv")
df.head()

Now we'll create tools that let the agent query this data. In LangGraph, tools are standard Python functions decorated with `@tool`. The function signature and docstring tell the LLM everything it needs.

::: {.callout-note}
**@tool**: Decorator that converts a function into a LangChain tool. The docstring becomes the tool description; parameter names and type hints define the schema.

**Args section**: Must be explicitly documented for each parameter. LangGraph parses this to generate the JSON schema the LLM sees.
:::

In [ ]:
import io
from langchain_core.tools import tool
from pandasql import sqldf

@tool
def inspect_data() -> str:
    """Get a concise summary of the dataset's structure, including column names, non-null values, and data types."""
    buffer = io.StringIO()
    df.info(buf=buffer)
    return buffer.getvalue()

The structure is minimal. LangGraph infers everything from the function:

1. **Name**: Derived from function name (`inspect_data`)
2. **Description**: Extracted from the docstring
3. **Parameters**: Inferred from type hints and docstring Args section
4. **Return type**: Inferred from return type hint

This tool takes no inputs and returns the dataset schema. The agent calls it to discover column names and types before writing queries.

Let's add three more tools to give the agent more analytical capabilities:

In [ ]:
#| code-fold: true
@tool
def query_data(sql_query: str) -> str:
    """Query the fish dataset using SQL. The table is called 'df'. Use inspect_data first to see available columns. Use find_correlations to find correlations between columns.

    Args:
        sql_query: SQL query to execute (use 'df' as table name)
    """
    result = sqldf(sql_query, globals())
    return result.to_string()

@tool
def find_correlations(columns: list[str]) -> str:
    """Calculate the correlation matrix for a list of numeric columns in the fish dataset.

    Args:
        columns: A list of column names to calculate correlations for.
    """
    numeric_df = df[columns].select_dtypes(include=['number'])
    corr_matrix = numeric_df.corr()
    return corr_matrix.to_string()

@tool
def get_stats(column: str, species: str = None) -> str:
    """Get statistical summary (count, mean, std, min, max) for a specific column and optionally filter by species.

    Args:
        column: Column name to analyze
        species: Species to filter by (optional)
    """
    data = df
    if species:
        data = df[df["Species"] == species]

    stats = data[column].describe()
    prefix = f"Stats for {column}"
    if species:
        prefix += f" (Species: {species})"
    return f"{prefix}:\n{stats.to_string()}"

Now create the agent. LangGraph is centered on a **state graph**—a directed graph where nodes are functions and edges define transitions. This gives you explicit control over the ReAct loop.

::: {.callout-note}
**ChatOllama**: LangChain's Ollama integration. Supports tool calling via the model's native API.

**create_react_agent**: Factory function that builds a standard ReAct graph. It defines three nodes: (1) call the LLM, (2) execute tools, (3) check if done.

**recursion_limit**: Maximum graph iterations. Equivalent to `max_steps` in smolagents.
:::

In [ ]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

model = ChatOllama(
    model="glm-4.6:cloud",
    base_url="http://localhost:11434"
)

tools = [
    inspect_data,
    query_data,
    find_correlations,
    get_stats
]

agent = create_react_agent(model, tools)

Run the agent and watch it autonomously choose which tools to use.

In [ ]:
query = "Which fish species has the highest average weight?"
inputs = {"messages": [("user", query)]}

result = agent.invoke(inputs)

In [ ]:
#| code-fold: true
for message in result["messages"]:
    print(message.content)

The agent executes a ReAct loop. It reads the question "Which fish species has the highest average weight?" and realizes it needs to use SQL to group by species and calculate averages. LangGraph streams each step—you see the LLM's reasoning, the tool calls, and the observations in real time.

Let's try another query that requires multiple steps:

In [ ]:
query = "What distinctive physical characteristics stand out to identify Pike?"
inputs = {"messages": [("user", query)]}

result = agent.invoke(inputs)

In [ ]:
#| code-fold: true
for message in result["messages"]:
    print(message.content)

This demonstrates the power of the ReAct loop—the agent chains multiple observations together, building a solution step-by-step rather than attempting everything in one shot. Unlike smolagents' opaque loop, LangGraph exposes every state transition, making debugging straightforward.

::: {.callout-note collapse="true"}
## Why not more complex queries?

You might notice we're using simpler queries than you'd expect. This is intentional. Real-world agentic systems face reliability challenges:

- **JSON parsing errors**: Models sometimes generate malformed JSON or multiple JSON objects in one response
- **SQL limitations**: pandasql uses SQLite, which lacks advanced functions like `CORR()` for per-group correlations
- **Max steps**: Complex queries can hit iteration limits before completing

Production systems like Claude Code and Cursor handle these issues through better error recovery, more sophisticated prompting, and custom tool implementations. For learning purposes, we focus on simple queries that reliably demonstrate the ReAct pattern.
:::

# The Takeaway

This is the entire architecture of Google Antigravity, Claude Code, and Cursor—just scaled with better tools (file editing, terminal commands, web browsing) and better orchestration (parallel agents, verification artifacts).